In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
from matplotlib import pyplot as plt

In [94]:
def readBadgesData(path):
    data = {
        'plus': [],
        'firstname': [],
        'lastname': []
    }
    
    with open(path, 'r') as f:
        line = f.readline()
        while line:
            entry = line.split()
            if len(entry) == 3 and len(entry[1]) >= 3 and len(entry[-1]) >= 3:
                data['plus'].append(True if entry[0] == '+' else False)
                data['firstname'].append(entry[1].lower())
                data['lastname'].append(entry[-1].lower())
            
            line = f.readline()
    
    return pd.DataFrame(data)

df = readBadgesData('./badges.txt')

In [95]:
df.describe()

,plus,firstname,lastname
count,219,219,219
unique,2,182,215
top,True,david,gordon
freq,160,9,2


In [96]:
df.sample(10)

,plus,firstname,lastname
116,True,wai,lam
70,True,geoffrey,gordon
102,False,efim,kinber
202,True,paul,utgoff
71,True,jonathan,gratch
74,True,marko,grobelnik
4,False,chidanand,apte
166,True,huw,roberts
64,True,yolanda,gil
180,True,mark,schwabacher


In [97]:
from string import ascii_lowercase

def getLetterCounts(name):
    letters_list = list(0 for _ in range(len(ascii_lowercase)))
    for letter in name:
        if letter in ascii_lowercase:
            letters_list[ord(letter) - ord('a')] += 1
    
    return letters_list

VOWELS = ['a', 'e', 'i', 'o', 'u', 'y']

def getLettersType(name, maxlength):
    """1 if vower, -1 if consonant"""
    assert len(name) <= maxlength
    
    arr = list(0 for _ in range(maxlength))
    for i, letter in enumerate(name):
        if letter in VOWELS:
            arr[i] = 1
        else:
            arr[i] = -1
    return arr

In [98]:
max_name_length = df['firstname'].apply(len).max()
max_name_length

11

In [99]:
from sklearn.tree import DecisionTreeClassifier

firstnameLetterType = np.array(list(df['firstname'].apply(getLettersType, maxlength=max_name_length)))

X = firstnameLetterType
feature_names = ['name[%d]' % i for i in range(max_name_length)] 
Y = df['plus'].values

clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=2, presort=True)
# clf_tree = DecisionTreeClassifier(max_depth=5)
clf_tree.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=True, random_state=None,
            splitter='best')

In [100]:
# from https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb
import pydotplus # pip install pydotplus
from sklearn.tree import export_graphviz

def tree_graph_to_png(tree, feature_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, 
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)  
    graph.write_png(png_file_to_save)

In [101]:
tree_graph_to_png(tree=clf_tree, feature_names=feature_names, 
                  png_file_to_save='./tree1.png')

![](./tree1.png)

In [102]:
df[~df['plus']].count()

plus         59
firstname    59
lastname     59
dtype: int64

In [103]:
a = np.c_[df.index.values.reshape(-1, 1), Y, X] 
a = a[a[:, 1] == 0] # sign is minus
a = a[a[:, 3] == 1] # second letter is a vowel
# df.iloc[a[]]
df.iloc[a[:, 0]]

,plus,firstname,lastname
1,False,myriam,abramson
103,False,jyrki,kivinen
114,False,eyal,kushilevitz
